In [28]:
import re
import pandas as pd
import tqdm, time

#### 1. generate a txt file with format chrN:12345678-12345678 using chromosome and hg19_position from file all_variants_and_proxies.csv

In [3]:
df = pd.read_csv('~/Documents/Materials/iron/all_variants_and_proxies.csv')
# df['format'] = 'chr'+df.chromosome.astype(str)+':'+df.hg19_position.astype(str)+'-'+df.hg19_position.astype(str)
# df['format'].to_csv('~/Documents/Projects/iron/variants_hg19.txt', index=False) 

#### 2. convert hg19 coordinates to hg38 coordinates using liftover tool: 
https://genome.ucsc.edu/cgi-bin/hgLiftOver

#### 3. rename the file downloaded from the tool as variants_hg19.bed

#### 4. create a list of format that can be recognized in finngen

In [7]:
df0 = pd.read_csv('~/Documents/Materials/iron/variants_hg38.bed', header=None) 
df['hg38_position'] = df0[0].str.extract('\:(\d+)\-') 
df['format'] = 'chr'+df.chromosome.astype(str)+''+df.hg38_position+''+df.ea+''+df.nea 
df['format_'] = 'chr'+df.chromosome.astype(str)+''+df.hg38_position+''+df.nea+''+df.ea 
df.to_csv('~/Documents/Materials/iron/all_variants_and_proxies.csv', index=False)

#### 5. scrape off EAF from https://results.finngen.fi/ using rsid
e.g. Given rs1961437,

```head = '22:37114993:A:G (rs1961437)'``` 

```str_eaf = 'AF 9.1e-1(ranges from 9.1e-1 to 9.1e-1 across all phenotypes)'```

```eaf = 0.91```

In [33]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
browser = webdriver.Chrome(Service('/Users/feiwang/Documents/Projects/chromedriver'))
browser.get('https://results.finngen.fi/')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  after removing the cwd from sys.path.


In [96]:
# create finn_format call according to the snp format on https://results.finngen.fi/
df['web_format'] = df.chromosome.astype(str)+':'+df.hg38_position.astype(str)+':'+df.ea+':'+df.nea

In [160]:
finngen_format_list, finngen_af_list = [], []

for i in tqdm.tqdm(df.rsid.tolist()):
#     browser.get('https://results.finngen.fi/variant/'+1:22227885-T-C)
    # there are 9 SNPs without rsid in the file
    if type(i) == float:
        finngen_format_list.append('id isna')
        finngen_eaf_list.append('id isna')
    else:
        browser.find_element(By.XPATH, '//*[@id="navbar-searchbox-input"]').send_keys(i)
        browser.find_element(By.XPATH, '//*[@id="navbar-searchbox-input"]').send_keys(Keys.RETURN)
        time.sleep(5)
        try:
            head = browser.find_element(By.XPATH, '/html/body/div/div[1]/div/h1').text
        except:
    #         err = browser.find_element_by_xpath('//*[@id="reactEntry"]/div/div/div/p[1]').text
            head = 'id notfound'
        if head == 'id notfound':
            eaf = 'id notfound'
        else:
            try:
                str_af = browser.find_element(By.XPATH, '//*[@id="maf-range"]/p').text
                af = float(re.findall('AF ([\d|\.|e|\-]+)\(ranges', str_eaf)[0])
            except:
                af = 'eaf notfound'

        finngen_format_list.append(head)
        finngen_af_list.append(af)
        time.sleep(1)

  0%|          | 0/1039 [00:00<?, ?it/s]/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  # Remove the CWD from sys.path while we load stuff.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  # This is added back by InteractiveShellApp.init_path()
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:22: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
100%|██████████| 1039/1039 [1:50:57<00:00

In [169]:
df['finn_format'] = finngen_format_list
df['finn_af'] = finngen_af_list

In [198]:
df['finn_rsid'] = df.finn_format.str.extract(' \((rs\d+)\)')#('^(\d{1,2})\:')
df['finn_ref'] = df.finn_format.str.extract(':(\w+):\w+ \(')
df['finn_alt'] = df.finn_format.str.extract(':\w+:(\w+) \(')

In [199]:
df['match'] = df.finn_rsid == df.rsid

In [204]:
for i, row in df[(df.match == False)&(df.finn_rsid.isna())&(df.finn_format != 'id notfound')].iterrows():
    id_list = re.findall('(rs\d+)',row.finn_format)
    if row.rsid in id_list:
        df.loc[i,'finn_rsid'] = row.rsid
        df.loc[i,'match'] = True

In [219]:
df[(df.match == False)][['chromosome', 'hg19_position', 'rsid', 'ea', 'nea', 'eaf', 
    'finn_format', 'finn_af', 'finn_rsid', 'finn_ref', 'finn_alt', 'match']]

,chromosome,hg19_position,rsid,ea,nea,eaf,finn_format,finn_af,finn_rsid,finn_ref,finn_alt,match
3,1,66073952,rs6664374,T,C,0.348560,id notfound,id notfound,NaN,NaN,NaN,False
27,1,66106469,rs1938495,G,A,0.371530,id notfound,id notfound,NaN,NaN,NaN,False
35,1,66110878,rs61783272,G,T,0.364930,id notfound,id notfound,NaN,NaN,NaN,False
38,1,66112528,rs12130672,G,A,0.372340,id notfound,id notfound,NaN,NaN,NaN,False
39,1,66112676,rs12127241,G,T,0.370550,5:41027060:T:C (rs1212724127),0.00021,rs1212724127,T,C,False
...,...,...,...,...,...,...,...,...,...,...,...,...
3120,19,50097547,rs7251877,G,A,0.234650,id notfound,id notfound,NaN,NaN,NaN,False
3137,20,39103919,rs529408822,C,CAA,0.088731,id notfound,id notfound,NaN,NaN,NaN,False
3168,22,37488558,rs2080704,G,A,0.024627,id notfound,id notfound,NaN,NaN,NaN,False
3169,22,37488573,rs140851959,A,G,0.024081,id notfound,id notfound,NaN,NaN,NaN,False


In [221]:
df[['chromosome', 'hg19_position', 'rsid', 'ea', 'nea', 'eaf', 'finn_format', 'finn_af', 
    'finn_rsid', 'finn_ref', 'finn_alt', 'match']].to_csv('~/Documents/Materials/iron/all_variants_and_proxies_finn_af.csv')

#### 6. upload the file to green bucket: 
https://console.cloud.google.com/storage/browser/fg-production-sandbox-6_green

gsutil cp ~/Documents/Materials/iron/all_variants_and_proxies.csv gs://fg-production-sandbox-6_green/FeiyiWang